# 🧪 Test Interactivo del Grafo V2.1

**Versión:** V2.1 con Reglas Determinísticas  
**Fecha:** 2025-10-14

---

## 📋 Características del Sistema

### **Router**
- ✅ Top-K dinámico: `score2 >= score1 - 0.10` → K=2, sino K=1
- ✅ Tracking: visited, hops, MAX_HOPS=2

### **Parallel Executor**
- ✅ STOP_AFTER = 0.85: Cancela si resolved >= 0.85
- ✅ TOOL_TIMEOUT = 2s por tool
- ✅ BUDGET_TOOLS_PER_TURN = 6 tools máximo

### **Aggregator**
- ✅ Umbrales calibrados: talent=0.75, platform=0.70, business=0.70, content=0.68
- ✅ common_graph nunca ganador final
- ✅ MAX_HOPS = 2 re-routings

### **Clarifier**
- ✅ Loop-safe: MAX_HOPS=2
- ✅ Máximo 2 campos de clarificación

## 1. Setup - Imports y Configuración

In [1]:
from src.strands.main_router.graph import process_question_advanced

# Uso simple
result = await process_question_advanced(
    question="peliculas de coppola",
    max_hops=2
)

print(result['answer'])

Loading validation data from platform_name_iso

🎯 ADVANCED ROUTER
📝 Pregunta: peliculas de coppola
{
  "primary": "TALENT",
  "confidence": 0.90,
  "candidates": [
    {"category": "TALENT", "confidence": 0.90}
  ]
}[ROUTER] ✅ Grafo seleccionado: talent
[ROUTER] 📊 Confidence: 0.90
[ROUTER] 🔢 Candidatos: 1


🔍 VALIDATION PREPROCESSOR
📝 Pregunta: peliculas de coppola
[VALIDATION] 🤖 Ejecutando validación con LLM...
Para validar los directores de Coppola, usaré la herramienta validate_director:
Tool #1: validate_director

🔍 SQL QUERY EJECUTADA
📝 Operación: director exact search
📄 Query:

WITH q AS (SELECT %s::text AS s)
SELECT 
  d.id, 
  d.name,
  t.n_titles
FROM ms.directors d
CROSS JOIN q
LEFT JOIN LATERAL (
  SELECT COUNT(*)::integer AS n_titles
  FROM ms.directed_by db 
  WHERE db.director_id = d.id
) t ON TRUE
WHERE d.name ILIKE q.s
ORDER BY t.n_titles DESC NULLS LAST, d.name ASC
LIMIT 25

🔧 Parámetros: ('Coppola',)

✅ Query retornó 0 filas en 1.010s


🔍 SQL QUERY EJECUTADA
📝 Operaci

In [2]:
# Uso simple
result = await process_question_advanced(
    question="1",
    max_hops=2
)

print(result['answer'])


🎯 ADVANCED ROUTER
📝 Pregunta: 1
{
  "primary": "COMMON",
  "confidence": 0.75,
  "candidates": [
    {"category": "COMMON", "confidence": 0.75},
    {"category": "PLATFORM", "confidence": 0.5}
  ]
}[ROUTER] ✅ Grafo seleccionado: common
[ROUTER] 📊 Confidence: 0.75
[ROUTER] 🔢 Candidatos: 2
[ROUTER] ⏭️  Validación no requerida


🔍 VALIDATION PREPROCESSOR
📝 Pregunta: 1
[VALIDATION] ⏭️  Validación no requerida para este grafo, saltando...

🎬 DOMAIN GRAPH EXECUTOR
[DOMAIN] Ejecutando: common
[SUPERVISOR] Evaluando estado... tools=0, task=None
[SUPERVISOR] Primera iteracion, necesita clasificacion
ADMIN
[ADMIN NODE]
Question: 1
Current state:
   • Task: admin
   • Previous tool calls: 0
   • Accumulated data: 0 characters

[ROUTING] Selecting tool...
   🔍 Router LLM analizando pregunta...
   📋 Tools disponibles: build_sql, run_sql_adapter, validate_intent
admin_validate_intent   💡 LLM sugiere: admin_validate_intent
   ⏱️  Tiempo de respuesta: 1.07s
   ❌ Tool inválida: 'admin_validate_intent'

In [1]:
import sys
import asyncio
from pathlib import Path
import json
from datetime import datetime

# Agregar src al path si no está
if str(Path.cwd()) not in sys.path:
    sys.path.insert(0, str(Path.cwd()))

# Imports del sistema
from src.strands.main_router.graph import create_main_graph, process_question
from src.strands.main_router.aggregator import THRESHOLD_BY_NODE, MAX_HOPS
from src.strands.main_router.parallel_executor import TOOL_TIMEOUT, BUDGET_TOOLS_PER_TURN, STOP_AFTER

print("✅ Imports completados")
print(f"\n📊 Configuración:")
print(f"  • Umbrales: {THRESHOLD_BY_NODE}")
print(f"  • MAX_HOPS: {MAX_HOPS}")
print(f"  • TOOL_TIMEOUT: {TOOL_TIMEOUT}s")
print(f"  • BUDGET: {BUDGET_TOOLS_PER_TURN} tools")
print(f"  • STOP_AFTER: {STOP_AFTER}")

Loading validation data from platform_name_iso
✅ Imports completados

📊 Configuración:
  • Umbrales: {'platform_graph': 0.7, 'business_graph': 0.7, 'talent_graph': 0.75, 'content_graph': 0.68, 'common_graph': None}
  • MAX_HOPS: 2
  • TOOL_TIMEOUT: 2.0s
  • BUDGET: 6 tools
  • STOP_AFTER: 0.85


## 2. Crear Grafo

In [2]:
# Crear grafo
try:
    graph = create_main_graph()
    print("✅ Grafo creado correctamente")
except Exception as e:
    print(f"❌ Error al crear grafo: {e}")
    raise

✅ Grafo creado correctamente


## 3. Función Helper para Testear

In [3]:
def print_result(result, question):
    """
    Imprime resultado de manera legible.
    """
    print("\n" + "="*80)
    print(f"📝 PREGUNTA: {question}")
    print("="*80)
    
    # Respuesta
    answer = result.get("answer", "Sin respuesta")
    print(f"\n💬 RESPUESTA:\n{answer}")
    
    # Metadata del router
    print(f"\n🎯 ROUTER:")
    routing_scores = result.get("routing_scores", {})
    if routing_scores:
        sorted_scores = sorted(routing_scores.items(), key=lambda x: x[1], reverse=True)
        for graph, score in sorted_scores[:3]:
            print(f"  • {graph}: {score:.2f}")
    
    selected = result.get("selected_candidates", [])
    if selected:
        print(f"  ✅ Seleccionados: {[f'{g}({s:.2f})' for g, s in selected]}")
    
    # Metadata del aggregator
    print(f"\n⚖️ AGGREGATOR:")
    decision = result.get("aggregator_decision", "N/A")
    print(f"  • Decisión: {decision}")
    
    winning_node = result.get("winning_node")
    if winning_node:
        confidence = result.get("final_confidence", 0)
        threshold = result.get("threshold_used", 0)
        print(f"  • Ganador: {winning_node}")
        print(f"  • Confidence: {confidence:.3f} >= {threshold:.3f}")
    
    # Telemetría
    print(f"\n📊 TELEMETRÍA:")
    tools_used = result.get("tools_used", 0)
    budget_exhausted = result.get("budget_exhausted", False)
    print(f"  • Tools usados: {tools_used}/{BUDGET_TOOLS_PER_TURN}")
    print(f"  • Budget agotado: {budget_exhausted}")
    
    cancelled = result.get("cancelled_branch")
    if cancelled:
        print(f"  • Branch cancelado: {cancelled} (STOP_AFTER)")
    
    lat_parallel = result.get("lat_parallel")
    lat_aggregator = result.get("lat_aggregator")
    if lat_parallel:
        print(f"  • Latencia parallel: {lat_parallel:.3f}s")
    if lat_aggregator:
        print(f"  • Latencia aggregator: {lat_aggregator:.3f}s")
    
    # Re-routing
    hops = result.get("rerouting_count", 0)
    visited = result.get("visited_graphs", [])
    print(f"  • Hops: {hops}/{MAX_HOPS}")
    if visited:
        print(f"  • Visited: {visited}")
    
    # Clarificación
    needs_clarification = result.get("needs_clarification", False)
    if needs_clarification:
        reason = result.get("clarification_reason", "unknown")
        print(f"\n❓ CLARIFICACIÓN:")
        print(f"  • Razón: {reason}")
    
    print("\n" + "="*80)

print("✅ Helper function definida")

✅ Helper function definida


## 4. Tests de Preguntas

### 📝 Categorías de Preguntas

1. **TALENT** - Actores, directores, filmografías
2. **CONTENT** - Metadata de títulos (año, género, duración)
3. **PLATFORM** - Disponibilidad, dónde ver
4. **BUSINESS** - Precios, rankings, popularidad
5. **COMMON** - Estadísticas, administración

### Test 1: Pregunta TALENT (Alta Confianza)

In [4]:
# Pregunta sobre director (debería ir a talent_graph)
question = "Informacion de Inception"

result = await process_question(question)
print_result(result, question)


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: Informacion de Inception

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): N/A
  • Tipos de entidad: N/A
  • Tokens: 3


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: Informacion de Inception
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.7,
  "talent": 0.8,
  "content": 0.9,
  "platform": 0.6,
  "common": 0.3
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.7,\n  "talent": 0.8,\n  "content": 0.9,\n  "platform": 0.6,\n  "common": 0.3\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.7,\n  "talent": 0.8,\n  "content": 0.9,\n  "platform": 0.6,\n  "common": 0.3\n}

[SCORES] Puntuación de candidatos:
  • content: 0.60
  • talent: 0.40
  • business: 0.20
  • platform: 0.20
  • common: 0.10

[ROUTER] Top-K = 1 (diff=0.20 > 0.10)

[SELECTED] Candidatos selec

In [4]:
# Pregunta sobre director (debería ir a talent_graph)
question = "¿Qué películas ha dirigido Christopher Nolan?"

result = await process_question(question)
print_result(result, question)


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: ¿Qué películas ha dirigido Christopher Nolan?

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): ['AR', 'BO', 'BR', 'CL', 'CO', 'CR', 'CU', 'DO', 'EC', 'GT', 'HN', 'MX', 'NI', 'PA', 'PE', 'PR', 'PY', 'SV', 'UY', 'VE']
  • Tipos de entidad: movie
  • Tokens: 6


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: ¿Qué películas ha dirigido Christopher Nolan?
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.2,
  "talent": 0.9,
  "content": 0.6,
  "platform": 0.1,
  "common": 0.0
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.2,\n  "talent": 0.9,\n  "content": 0.6,\n  "platform": 0.1,\n  "common": 0.0\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.2,\n  "talent": 0.9,\n  "content": 0.6,\n  "platform": 0.1,\n  "common": 0.0\n}

[SCORES] Puntuación de candidat

### Test 2: Pregunta PLATFORM (Alta Confianza)

In [5]:
# Pregunta sobre disponibilidad (debería ir a platform_graph)
question = "¿Dónde puedo ver Stranger Things en Argentina?"

result = await process_question(question)
print_result(result, question)


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: ¿Dónde puedo ver Stranger Things en Argentina?

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
  • Tipos de entidad: country
  • Tokens: 7


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: ¿Dónde puedo ver Stranger Things en Argentina?
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.2,
  "talent": 0.1,
  "content": 0.3,
  "platform": 0.8,
  "common": 0.0
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.2,\n  "talent": 0.1,\n  "content": 0.3,\n  "platform": 0.8,\n  "common": 0.0\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.2,\n  "talent": 0.1,\n  "content": 0.3,\n  "platform": 0.8,\n  "co

### Test 3: Pregunta BUSINESS (Precios)

In [6]:
# Pregunta sobre precios (debería ir a business_graph)
question = "¿Cuánto cuesta Netflix en Argentina?"

result = await process_question(question)
print_result(result, question)


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: ¿Cuánto cuesta Netflix en Argentina?

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
  • Tipos de entidad: price, platform
  • Tokens: 5


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: ¿Cuánto cuesta Netflix en Argentina?
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.8,
  "talent": 0.0,
  "content": 0.0,
  "platform": 0.9,
  "common": 0.0
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.8,\n  "talent": 0.0,\n  "content": 0.0,\n  "platform": 0.9,\n  "common": 0.0\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.8,\n  "talent": 0.0,\n  "content": 0.0,\n  "platform": 0.9,\n  "common": 0.0\n

### Test 4: Pregunta CONTENT (Metadata)

In [7]:
# Pregunta sobre metadata (debería ir a content_graph)
question = "¿De qué año es la película Inception?"

result = await process_question(question)
print_result(result, question)


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: ¿De qué año es la película Inception?

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): ['AR', 'BO', 'BR', 'CL', 'CO', 'CR', 'CU', 'DO', 'EC', 'GT', 'HN', 'MX', 'NI', 'PA', 'PE', 'PR', 'PY', 'SV', 'UY', 'VE']
  • Tipos de entidad: movie
  • Tokens: 7


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: ¿De qué año es la película Inception?
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.1,
  "talent": 0.2,
  "content": 0.8,
  "platform": 0.2,
  "common": 0.1
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.1,\n  "talent": 0.2,\n  "content": 0.8,\n  "platform": 0.2,\n  "common": 0.1\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.1,\n  "talent": 0.2,\n  "content": 0.8,\n  "platform": 0.2,\n  "common": 0.1\n}

[SCORES] Puntuación de candidatos:
  • content:

### Test 5: Pregunta Ambigua (Top-K = 2)

In [8]:
# Pregunta ambigua que podría ir a múltiples grafos
question = "¿Cuánto cuesta?"

result = await process_question(question)
print_result(result, question)

# Debería mostrar:
# - Top-K = 2 (scores similares)
# - Posible re-routing si confidence < threshold


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: ¿Cuánto cuesta?

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
  • Tipos de entidad: price
  • Tokens: 2


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: ¿Cuánto cuesta?
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.9,
  "talent": 0.1,
  "content": 0.2,
  "platform": 0.2,
  "common": 0.1
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.9,\n  "talent": 0.1,\n  "content": 0.2,\n  "platform": 0.2,\n  "common": 0.1\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.9,\n  "talent": 0.1,\n  "content": 0.2,\n  "platform": 0.2,\n  "common": 0.1\n}

[SCORES] Puntuación de candidatos:
  • business: 

### Test 6: Pregunta con Nombre Ambiguo

In [9]:
# Pregunta con nombre que podría tener múltiples matches
question = "¿Qué películas ha hecho Nolan?"

result = await process_question(question)
print_result(result, question)

# Debería mostrar:
# - talent_graph con alta confianza
# - Posible ambiguous si hay múltiples "Nolan" (Christopher, Jonathan)


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: ¿Qué películas ha hecho Nolan?

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): ['AR', 'BO', 'BR', 'CL', 'CO', 'CR', 'CU', 'DO', 'EC', 'GT', 'HN', 'MX', 'NI', 'PA', 'PE', 'PR', 'PY', 'SV', 'UY', 'VE']
  • Tipos de entidad: movie
  • Tokens: 5


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: ¿Qué películas ha hecho Nolan?
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.2,
  "talent": 0.9,
  "content": 0.6,
  "platform": 0.1,
  "common": 0.0
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.2,\n  "talent": 0.9,\n  "content": 0.6,\n  "platform": 0.1,\n  "common": 0.0\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.2,\n  "talent": 0.9,\n  "content": 0.6,\n  "platform": 0.1,\n  "common": 0.0\n}

[SCORES] Puntuación de candidatos:
  • talent: 0.90
  • conte

## 5. Test Personalizado

Escribe tu propia pregunta aquí:

In [10]:
# 👇 Escribe tu pregunta aquí
question = "Tu pregunta aquí"

result = await process_question(question)
print_result(result, question)


🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: Tu pregunta aquí

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): N/A
  • Tipos de entidad: N/A
  • Tokens: 3


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: Tu pregunta aquí
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.3,
  "talent": 0.7,
  "content": 0.8,
  "platform": 0.4,
  "common": 0.0
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.3,\n  "talent": 0.7,\n  "content": 0.8,\n  "platform": 0.4,\n  "common": 0.0\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.3,\n  "talent": 0.7,\n  "content": 0.8,\n  "platform": 0.4,\n  "common": 0.0\n}

[SCORES] Puntuación de candidatos:
  • content: 0.60
  • talent: 0.40
  • business: 0.20
  • platform: 0.20
  • common: 0.10

[ROUTER] Top-K = 1 (diff=0.20 > 0.10)

[SELECTED] Candidatos seleccionados: 1
  ✅ 

## 6. Análisis de Múltiples Preguntas

In [11]:
# Lista de preguntas para testear en batch
test_questions = [
    "¿Qué películas ha dirigido Steven Spielberg?",
    "¿Dónde puedo ver Breaking Bad?",
    "¿Cuánto cuesta Disney+ en México?",
    "¿De qué año es The Matrix?",
    "¿Qué plataformas tienen Game of Thrones?",
    "¿Quién actuó en Titanic?",
    "¿Cuál es la serie más popular?",
]

results = []

for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"TEST {i}/{len(test_questions)}")
    print(f"{'='*80}")
    
    try:
        result = await process_question(q)
        print_result(result, q)
        
        # Guardar para análisis
        results.append({
            "question": q,
            "winning_node": result.get("winning_node"),
            "confidence": result.get("final_confidence"),
            "decision": result.get("aggregator_decision"),
            "hops": result.get("rerouting_count", 0)
        })
    except Exception as e:
        print(f"❌ Error: {e}")
        results.append({
            "question": q,
            "error": str(e)
        })

print(f"\n\n{'='*80}")
print("📊 RESUMEN DE TESTS")
print(f"{'='*80}")
print(f"Total preguntas: {len(results)}")
print(f"\nResultados por nodo:")

from collections import Counter
node_counts = Counter([r.get("winning_node") for r in results if "winning_node" in r])
for node, count in node_counts.most_common():
    print(f"  • {node}: {count}")


TEST 1/7

🔍 LIGHTWEIGHT PREPROCESSOR - Normalización Barata
📝 Pregunta: ¿Qué películas ha dirigido Steven Spielberg?

[PREPROCESSING] Contexto extraído:
  • País (ISO-2): ['AR', 'BO', 'BR', 'CL', 'CO', 'CR', 'CU', 'DO', 'EC', 'GT', 'HN', 'MX', 'NI', 'PA', 'PE', 'PR', 'PY', 'SV', 'UY', 'VE']
  • Tipos de entidad: movie
  • Tokens: 6


🎯 UNIFIED ROUTER - Puntuación de Candidatos
📝 Pregunta: ¿Qué películas ha dirigido Steven Spielberg?
[ROUTER] Visited: set(), Hops: 0/2
[ROUTER] 🤖 Puntuando candidatos con LLM...
{
  "business": 0.2,
  "talent": 0.9,
  "content": 0.6,
  "platform": 0.1,
  "common": 0.0
}{'role': 'assistant', 'content': [{'text': '{\n  "business": 0.2,\n  "talent": 0.9,\n  "content": 0.6,\n  "platform": 0.1,\n  "common": 0.0\n}'}]}
[ERROR] JSON inválido: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
[DEBUG] JSON extraído: {\n  "business": 0.2,\n  "talent": 0.9,\n  "content": 0.6,\n  "platform": 0.1,\n  "common": 0.0\n}

[SCORES] Puntuación de 

## 7. Exportar Resultados

In [12]:
# Exportar resultados a JSON
output_file = f"test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"✅ Resultados exportados a: {output_file}")

✅ Resultados exportados a: test_results_20251014_235426.json


## 8. Verificación de Reglas Determinísticas

In [13]:
# Verificar que las reglas se cumplen
print("🔍 VERIFICACIÓN DE REGLAS DETERMINÍSTICAS")
print("="*80)

# 1. Umbrales
print("\n1. Umbrales por nodo:")
for node, threshold in THRESHOLD_BY_NODE.items():
    if threshold is not None:
        print(f"  ✅ {node}: {threshold}")
    else:
        print(f"  🚫 {node}: None (nunca ganador)")

# 2. Presupuesto
print(f"\n2. Presupuesto y Timeout:")
print(f"  ✅ TOOL_TIMEOUT: {TOOL_TIMEOUT}s")
print(f"  ✅ BUDGET_TOOLS_PER_TURN: {BUDGET_TOOLS_PER_TURN}")
print(f"  ✅ STOP_AFTER: {STOP_AFTER}")

# 3. Loop-safe
print(f"\n3. Loop-safe:")
print(f"  ✅ MAX_HOPS: {MAX_HOPS}")
print(f"  ✅ MAX_CLARIFICATION_FIELDS: 2")

# 4. Análisis de resultados
if results:
    print(f"\n4. Análisis de resultados:")
    
    # Verificar que ningún common_graph ganó
    common_wins = [r for r in results if r.get("winning_node") == "common_graph"]
    if not common_wins:
        print(f"  ✅ common_graph nunca ganador (0/{len(results)})")
    else:
        print(f"  ❌ common_graph ganó {len(common_wins)} veces (ERROR)")
    
    # Verificar hops
    max_hops_used = max([r.get("hops", 0) for r in results])
    if max_hops_used <= MAX_HOPS:
        print(f"  ✅ Max hops respetado: {max_hops_used}/{MAX_HOPS}")
    else:
        print(f"  ❌ Max hops excedido: {max_hops_used} > {MAX_HOPS}")
    
    # Verificar confidence vs threshold
    threshold_violations = 0
    for r in results:
        node = r.get("winning_node")
        conf = r.get("confidence")
        if node and conf:
            threshold = THRESHOLD_BY_NODE.get(node, 0.65)
            if conf < threshold:
                threshold_violations += 1
    
    if threshold_violations == 0:
        print(f"  ✅ Todos los ganadores cumplen threshold (0 violaciones)")
    else:
        print(f"  ❌ {threshold_violations} ganadores no cumplen threshold")

print("\n" + "="*80)

🔍 VERIFICACIÓN DE REGLAS DETERMINÍSTICAS

1. Umbrales por nodo:
  ✅ platform_graph: 0.7
  ✅ business_graph: 0.7
  ✅ talent_graph: 0.75
  ✅ content_graph: 0.68
  🚫 common_graph: None (nunca ganador)

2. Presupuesto y Timeout:
  ✅ TOOL_TIMEOUT: 2.0s
  ✅ BUDGET_TOOLS_PER_TURN: 6
  ✅ STOP_AFTER: 0.85

3. Loop-safe:
  ✅ MAX_HOPS: 2
  ✅ MAX_CLARIFICATION_FIELDS: 2

4. Análisis de resultados:
  ✅ common_graph nunca ganador (0/7)
  ✅ Max hops respetado: 2/2
  ✅ Todos los ganadores cumplen threshold (0 violaciones)



## 9. Notas y Observaciones

Usa esta celda para agregar notas sobre los tests:

### 📝 Observaciones:

- [ ] Top-K dinámico funciona correctamente
- [ ] STOP_AFTER cancela branches cuando corresponde
- [ ] Umbrales calibrados son apropiados
- [ ] Clarifier es loop-safe
- [ ] Presupuesto se respeta

### 🐛 Issues encontrados:

1. ...
2. ...

### 💡 Mejoras sugeridas:

1. ...
2. ...